In [1]:
# Import des packages
import pandas as pd

C:\Users\tlp\Anaconda3\lib\site-packages\pandas\core\computation\expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
# Lecture des données
df = pd.read_csv(r'data_cleaned/corpus_cleaned_PAMPLEMOUSSE.csv',sep=";")
df["annee"] = df.Date.apply(lambda date: date[:4])

In [3]:
# Calcule de la df_distri de chaque journal par année
df_distri = df.groupby(['CJournal', 'annee']).size().reset_index(name='count')

# Calcule du le nombre total d'articles par journal et par année
total_articles = df_distri['count'].sum()

# Déterminantion du nombre d'articles qui doivent être sélectionnés 
# On vise 300 articles pour commencer
df_distri['nombre_sample'] = (df_distri['count'] / total_articles * 300).round().astype(int)

# Initialisation d'une liste pour stocker les articles sélectionnés
gs = []

# Sélection des articles par une boucle sur les journaux X années
for index, row in df_distri.iterrows():
    journal = row['CJournal']
    year = row['annee']
    nombre_sample = row['nombre_sample']
    
    filtered_df = df.loc[(df['CJournal'] == journal) & (df['annee'] == year)]
    
    selected_articles = filtered_df.sample(n=nombre_sample, random_state=42)
    
    gs.append(selected_articles)

# Assemblage final
df_gs = pd.concat(gs)

# Stockage du GS en csv
#df_gs.to_csv('data_cleaned/gs.csv', index=False)

In [4]:
# Stat desc du GS
df_gs[['CJournal', 'annee','Texte']].groupby(['CJournal', 'annee']).count()

Texte
CJournal   annee       
Croix      2015       4
           2016       4
           2017       2
           2018       6
           2019       3
           2020       2
           2021       2
           2022       1
Echos      2015      11
           2016      14
           2017       5
           2018       6
           2019       2
           2020       1
           2021       1
           2022       1
Figaro     2015      26
           2016      28
           2017      10
           2018      16
           2019       9
           2020       5
           2021       6
           2022       2
Libération 2015       5
           2016       6
           2017       3
           2018       7
           2019       4
           2020       2
           2021       2
           2022       1
Monde      2015      18
           2016      23
           2017      11
           2018      10
           2019       6
           2020       3
           2021       3
           2022       1
Parisien   2015       7
           2016       5
           2017       3
           2018       3
           2019       2
           2020       1
           2021       2

In [5]:
print(f"""
    Le GS initial compte {len(df_gs)} articles
""")


    Le GS initial compte 295 articles



### Complement au GS

Les résultats initiaux n'étaient pas concluant avec le Gold Standard intial.  
Nous avons donc décidé de le compléter avec une centaines d'articles supplémentaires pour améliorer les prévisions des modèles

In [18]:
# Lecture des données
gs = pd.read_csv('./data_cleaned/gold_standard.csv')
corpus = pd.read_csv(r'data_cleaned/corpus_cleaned_PAMPLEMOUSSE.csv',sep=";")

In [19]:
# Ne pas reprendre les mêmes articles
df_filtered = corpus[~(corpus.Texte.isin(gs.Texte.unique()))].copy()

In [20]:
df_filtered["annee"] = df_filtered.Date.apply(lambda x: x[:4])
# Calcule de la df_distri de chaque journal par année
df_distri = df_filtered.groupby(['CJournal', 'annee']).size().reset_index(name='count')

# Calcule du le nombre total d'articles par journal et par année
total_articles = df_distri['count'].sum()

# Déterminantion du nombre d'articles qui doivent être sélectionnés 
# On vise 300 articles pour commencer
df_distri['nombre_sample'] = (df_distri['count'] / total_articles * 100).round().astype(int)

# Initialisation d'une liste pour stocker les articles sélectionnés
gs = []

# Sélection des articles par une boucle sur les journaux X années
for index, row in df_distri.iterrows():
    journal = row['CJournal']
    year = row['annee']
    nombre_sample = row['nombre_sample']
    
    filtered_df = df.loc[(df['CJournal'] == journal) & (df['annee'] == year)]
    
    selected_articles = filtered_df.sample(n=nombre_sample, random_state=42)
    
    gs.append(selected_articles)

# Assemblage final
df_filtered_gs = pd.concat(gs)

# Stockage du GS complémentaire en csv
#df_filtered_gs.to_csv('data_cleaned/gold_standard_complement.csv', index=False)

In [23]:
# Stat desc du GS complet
GS = pd.concat([
    df_gs, df_filtered_gs
])[['CJournal', 'annee','Texte']]

GS.groupby(['CJournal', 'annee']).count()

Texte
CJournal   annee       
Croix      2015       5
           2016       5
           2017       3
           2018       8
           2019       4
           2020       3
           2021       3
           2022       1
Echos      2015      15
           2016      19
           2017       7
           2018       8
           2019       3
           2020       1
           2021       1
           2022       1
Figaro     2015      35
           2016      37
           2017      13
           2018      21
           2019      12
           2020       7
           2021       8
           2022       3
Libération 2015       7
           2016       8
           2017       4
           2018       9
           2019       5
           2020       3
           2021       3
           2022       1
Monde      2015      24
           2016      31
           2017      15
           2018      13
           2019       8
           2020       4
           2021       4
           2022       1
Parisien   2015       9
           2016       7
           2017       4
           2018       4
           2019       3
           2020       1
           2021       3

In [24]:
print(f"""
    Le GS final compte {len(GS)} articles
""")


    Le GS final compte 394 articles

